In [1]:
import pandas as pd
import numpy as np 
# import seaborn as sns
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
# from category_encoders import TargetEncoder, one_hot
from catboost import *
%matplotlib inline

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
0,0,2,908749,954,480270,935,71,79,53,89,...,51,100,87,3,61,48,43,79,329,0
1,1,1,438444,2162,486685,2154,32,39,48,49,...,39,83,91,6,57,94,15,89,581,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,7,52,37,50,69,60,89,10,548,0
3,3,1,625198,1508,66810,1474,7,1,34,73,...,39,27,55,97,83,1,88,87,555,1
4,4,2,228654,1202,542816,1196,53,35,16,80,...,35,95,50,55,10,49,12,68,651,0


# Data Prepairing

In [48]:
y = data.target
# X = data.drop(['target', 'id', 'X1', 'X3'], axis=1)
X = data.drop(['target', 'id'], axis=1)

In [49]:
army_count_1 = X['X5'] + X['X7'] + X['X9'] + X['X11'] + X['X13'] + X['X15'] + X['X17'] + X['X19']
army_count_2 = X['X6'] + X['X8'] + X['X10'] + X['X12'] + X['X14'] + X['X16'] + X['X18'] + X['X20']
# X['army_relation'] = list(map(lambda x: 1 if x > 1 else 0, army_count_1/army_count_2))
X['army_1'] = army_count_1
X['army_2'] = army_count_2
X['type_time'] = X['X0'] * X['X21']
X['pers1_time'] = X['X1'] * X['X21']
X['pers2_time'] = X['X3'] * X['X21']
X['rank_relation'] = list(map(lambda x: 1 if x > 1 else 0, X['X2']/(X['X4'])))

In [50]:
# first_player_army = ['X5', 'X7', 'X9', 'X11', 'X13', 'X15', 'X17', 'X19']
# second_player_army = ['X6', 'X8', 'X10', 'X12', 'X14', 'X16', 'X18', 'X20']

# for i in range(len(first_player_army)):
#     unit_1 = first_player_army[i]
#     unit_2 = second_player_army[i]
#     X[unit_1 + '-' + unit_2] = list(map(lambda x: 1 if x > 1 else 0, X[unit_1]/X[unit_2])) #round(100 * X[unit_1]/(X[unit_2] + 1)).astype(int)

In [51]:
X.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X14,X15,X16,X17,X18,X19,X20,X21,army_relation,rank_relation
0,2,908749,954,480270,935,71,79,53,89,15,...,100,87,3,61,48,43,79,329,0,1
1,1,438444,2162,486685,2154,32,39,48,49,100,...,83,91,6,57,94,15,89,581,1,1
2,1,596915,2066,711059,2081,43,21,58,94,88,...,52,37,50,69,60,89,10,548,0,0
3,1,625198,1508,66810,1474,7,1,34,73,6,...,27,55,97,83,1,88,87,555,0,1
4,2,228654,1202,542816,1196,53,35,16,80,72,...,95,50,55,10,49,12,68,651,0,1


In [52]:
# cat_features = list(range(0, X.shape[1] - 12))
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [53]:
print('Labels: {}'.format(set(y)))
print('Zero count = {}, One count = {}'.format(len(y) - sum(y), sum(y)))

Labels: {0, 1}
Zero count = 2252162, One count = 1380843


In [54]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True)

# Model training

In [55]:
# !rm 'catboost_info/snapshot.bkp'
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,
    verbose=10,
    random_seed=42,
    learning_rate=0.5,
    # save_snapshot=True,
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=1,
    l2_leaf_reg=5.0,
    early_stopping_rounds=20,
    one_hot_max_size=2,
    bootstrap_type='Bernoulli',
    subsample=0.5,
    random_strength=2,
    use_best_model=True,
    best_model_min_trees=30,
    rsm=0.5,
    leaf_estimation_iterations=5
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    plot=False
)

0:	learn: 0.6711872	test: 0.6709816	best: 0.6709816 (0)	total: 11.7s	remaining: 3h 14m 11s
10:	learn: 0.6585733	test: 0.6546949	best: 0.6546949 (10)	total: 1h 52m 52s	remaining: 7d 1h 8m 40s
20:	learn: 0.6564254	test: 0.6526938	best: 0.6526938 (20)	total: 1h 54m 38s	remaining: 3d 17h 4m 18s
30:	learn: 0.6554730	test: 0.6515552	best: 0.6515552 (30)	total: 1h 56m 13s	remaining: 2d 12h 32m 46s
40:	learn: 0.6546750	test: 0.6501978	best: 0.6501978 (40)	total: 1h 58m 10s	remaining: 1d 22h 4m 4s
50:	learn: 0.6536863	test: 0.6503496	best: 0.6501978 (40)	total: 1h 59m 53s	remaining: 1d 13h 10m 58s
60:	learn: 0.6534526	test: 0.6501415	best: 0.6501415 (60)	total: 2h 1m 18s	remaining: 1d 7h 7m 21s
70:	learn: 0.6531598	test: 0.6493028	best: 0.6493028 (70)	total: 2h 3m 11s	remaining: 1d 2h 51m 52s
80:	learn: 0.6530190	test: 0.6492173	best: 0.6492173 (80)	total: 2h 4m 36s	remaining: 23h 33m 48s
90:	learn: 0.6528700	test: 0.6491394	best: 0.6491394 (90)	total: 2h 6m 13s	remaining: 21h 50s
100:	learn: 0

# Feature Importance

In [56]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,X3,30.301919
1,X1,21.007065
2,X2,6.630050
3,X4,4.817018
4,X9,2.601869
5,X13,2.517036
6,X20,2.453401
7,X7,2.435469
8,X16,2.399470
9,X12,2.273118


# Hyperparameter tunning

# Saving Model

# Calculate predictions for the contest

In [57]:
data_test = pd.read_csv('test.csv')
data_test.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,0,1,1032217,1187,1011523,1172,57,100,1,80,...,79,45,18,35,10,80,81,16,0,547
1,1,2,1059033,2128,505263,2104,82,27,87,79,...,74,96,55,54,48,3,92,5,51,656
2,2,1,207787,1017,822639,1040,82,48,43,62,...,16,64,35,70,99,60,30,37,45,437
3,3,1,1095582,1449,32841,1424,71,43,11,98,...,1,7,41,94,26,52,45,81,1,589
4,4,2,963764,1202,489871,1182,11,62,16,55,...,26,60,37,7,1,80,48,50,52,340


In [58]:
X_test = data_test.drop(['id'], axis=1)
X_test.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21
0,1,1032217,1187,1011523,1172,57,100,1,80,25,...,79,45,18,35,10,80,81,16,0,547
1,2,1059033,2128,505263,2104,82,27,87,79,80,...,74,96,55,54,48,3,92,5,51,656
2,1,207787,1017,822639,1040,82,48,43,62,18,...,16,64,35,70,99,60,30,37,45,437
3,1,1095582,1449,32841,1424,71,43,11,98,80,...,1,7,41,94,26,52,45,81,1,589
4,2,963764,1202,489871,1182,11,62,16,55,88,...,26,60,37,7,1,80,48,50,52,340


In [60]:
test_army_count_1 = X_test['X5'] + X_test['X7'] + X_test['X9'] + X_test['X11'] + X_test['X13'] + X_test['X15'] + X_test['X17'] + X_test['X19']
test_army_count_2 = X_test['X6'] + X_test['X8'] + X_test['X10'] + X_test['X12'] + X_test['X14'] + X_test['X16'] + X_test['X18'] + X_test['X20']
X_test['army_relation'] = list(map(lambda x: 1 if x > 1 else 0, test_army_count_1/test_army_count_2))
X_test['rank_relation'] = list(map(lambda x: 1 if x > 1 else 0, X_test['X2']/(X_test['X4'])))

In [61]:
y_test = model.predict_proba(X_test)
y_test = y_test[:, 1]

# Prepare the submission

In [62]:
ans = pd.DataFrame(data=y_test, columns=["target"])
ans.reset_index(level=0, inplace=True)
ans.reset_index(drop=True, inplace=True)
ans = ans.rename(columns={'index':'id'})
ans.head()

,id,target
0,0,0.405511
1,1,0.352932
2,2,0.314297
3,3,0.386206
4,4,0.335869


In [63]:
ans.to_csv('submission.csv',index=False)